In [54]:
import pickle
import numpy as np
import pandas as pd
from os import path
import ta
from CSVUtils import *

In [14]:
DIR = "./output/306"
df = pickle.load(open(path.join(DIR,"BRZ_TW_NASDAQ-Selected_Trans-withleakage+RSI-_detailed-ModelNo-100-0.out"), "rb"))

In [15]:
df = pd.DataFrame(df)

In [17]:
df[]

ValueError: Item wrong length 4 instead of 1113.

In [50]:
pd.set_option('display.max_rows', df.shape[0]+1)
print(df[['date', 'action']][[sum(tmp)==0.0 for tmp in df['action']]])

           date                action
4    2015-01-20  [0.0, 0.0, 0.0, 0.0]
14   2015-02-03  [0.0, 0.0, 0.0, 0.0]
16   2015-02-05  [0.0, 0.0, 0.0, 0.0]
30   2015-03-06  [0.0, 0.0, 0.0, 0.0]
35   2015-03-13  [0.0, 0.0, 0.0, 0.0]
37   2015-03-17  [0.0, 0.0, 0.0, 0.0]
44   2015-03-26  [0.0, 0.0, 0.0, 0.0]
69   2015-05-06  [0.0, 0.0, 0.0, 0.0]
89   2015-06-05  [0.0, 0.0, 0.0, 0.0]
93   2015-06-11  [0.0, 0.0, 0.0, 0.0]
125  2015-07-31  [0.0, 0.0, 0.0, 0.0]
148  2015-09-02  [0.0, 0.0, 0.0, 0.0]
149  2015-09-03  [0.0, 0.0, 0.0, 0.0]
150  2015-09-04  [0.0, 0.0, 0.0, 0.0]
184  2015-10-29  [0.0, 0.0, 0.0, 0.0]
190  2015-11-09  [0.0, 0.0, 0.0, 0.0]
201  2015-11-25  [0.0, 0.0, 0.0, 0.0]
209  2015-12-08  [0.0, 0.0, 0.0, 0.0]
227  2016-01-07  [0.0, 0.0, 0.0, 0.0]
243  2016-02-02  [0.0, 0.0, 0.0, 0.0]
260  2016-03-09  [0.0, 0.0, 0.0, 0.0]
261  2016-03-10  [0.0, 0.0, 0.0, 0.0]
264  2016-03-15  [0.0, 0.0, 0.0, 0.0]
288  2016-04-22  [0.0, 0.0, 0.0, 0.0]
295  2016-05-04  [0.0, 0.0, 0.0, 0.0]
308  2016-05

In [56]:
rootDir = "./data"
df_namelist = {"high": "^BVSP_new", "mid": "^TWII_new", "low": "^IXIC_new"}
df_list = []
for key in df_namelist:
    fileName = df_namelist[key]+".csv"
    if df_namelist[key][-4:]=="_new":
        source = "done"
        # price_label = 'Actual Price'
        price_label = 'Price'

    else:
        source = "yahoo"
        price_label = 'Price'
    df = csv2df(rootDir, fileName, source = source)
    df = df.sort_values('Date').reset_index(drop=True)
    # Add TA Indicators
    # <1> EMA
    df['EMA'] = df[price_label].ewm(span=15).mean()
    # <2> MACD_diff
    df['MACD_diff'] = ta.trend.macd_diff(df[price_label], fillna=True)
    macd_direction = df['MACD_diff']/np.abs(df['MACD_diff']) # 1: No change, -1: Change Sign
    # <3> MACD_change
    df['MACD_change'] = (-1*macd_direction*macd_direction.shift(1)+1)/2 # 1: Change Sign, 0: No Change
    
    # <4> delta_time: How many days since the last trend change
    delta_time = [] 
    for i in df['MACD_change']:
        if len(delta_time) == 0:
            result = 0
        elif i==0:
            result = delta_time[-1]+1
        else: #Nan or 1
            result = 0
        delta_time.append(result)
    df['delta_time'] = delta_time

    # <5> RSI
    df['RSI'] = ta.momentum.RSIIndicator(df[price_label], fillna=True).rsi() 
    df['RSI'] /= df['RSI'][0]

    # Clean up all the nans
    df = df.dropna()
    df = df.reset_index(drop=True)
    
    df_list.append(df)

In [62]:
i=2
df_list[i][df_list[i]['Date'] >= pd.to_datetime("2017-02-15")]

,Date,Open,High,Low,Price,Vol,Change,FX Change,Cum FX Change,Actual Price,EMA,MACD_diff,MACD_change,delta_time,RSI
3929,2017-02-15,5777.899902,5821.950195,5776.700195,5819.439941,2083660000,0.006376,0,1,5819.439941,5682.584463,14.282284,0.0,9,0.834395
3930,2017-02-16,5823.009766,5835.149902,5796.709961,5814.899902,1943600000,-0.000780,0,1,5814.899902,5699.123893,13.684485,0.0,10,0.822069
3931,2017-02-17,5807.310059,5838.580078,5800.799805,5838.580078,1875580000,0.004072,0,1,5838.580078,5716.555916,13.456473,0.0,11,0.835702
3932,2017-02-21,5850.200195,5867.890137,5847.500000,5865.950195,1935110000,0.004688,0,1,5865.950195,5735.230201,13.677001,0.0,12,0.850007
3933,2017-02-22,5857.560059,5864.419922,5848.279785,5860.629883,1872330000,-0.000907,0,1,5860.629883,5750.905161,12.049708,0.0,13,0.834792
3934,2017-02-23,5866.819824,5866.959961,5809.549805,5835.509766,1879670000,-0.004286,0,1,5835.509766,5761.480737,8.058499,0.0,14,0.765149
3935,2017-02-24,5802.330078,5845.310059,5800.549805,5845.310059,1683280000,0.001679,0,1,5845.310059,5771.959402,5.005031,0.0,15,0.773102
3936,2017-03-01,5874.859863,5911.790039,5865.790039,5904.029785,2210360000,0.013491,0,1,5904.029785,5788.468200,5.809906,0.0,16,0.813790
3937,2017-03-02,5897.009766,5897.009766,5856.299805,5861.220215,2040770000,-0.007251,0,1,5861.220215,5797.562202,2.498318,0.0,17,0.713355
3938,2017-03-03,5854.270020,5870.750000,5841.470215,5870.750000,1852130000,0.001626,0,1,5870.750000,5806.710677,0.114691,0.0,18,0.721592


In [67]:
np.isinf(df)

TypeError: ufunc 'isinf' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [70]:
df[['Open', 'High']].values.sum()

32102030.477789003

In [74]:
abs(np.array([np.inf, -np.inf])).sum() == np.inf

True